# AutoGen AutoChat Agent Tutorial

## Overview

In this section, we explore AutoGen AutoChat, a powerful framework for building conversational AI agents that can interact with users and perform various tasks.

**AutoChat** is a component of the AutoGen framework that enables the creation of intelligent chat agents capable of:
- Processing natural language queries
- Executing complex tasks through conversation
- Maintaining context across interactions
- Integrating with various language models

We use an `autogen_agentchat.agents.AssistantAgent` to explore the core concepts of agents and demonstrate how they handle message processing and task execution.

## Key Concepts

### Core Components

- **AssistantAgent**: The primary agent class that handles user interactions and task execution
- **TextMessage**: The fundamental message type for text-based communication
- **TaskResult**: Contains the complete conversation history and results from agent execution
- **Response**: Lower-level response object returned by the `on_messages` method

### Message Flow

The typical conversation flow follows this pattern:
1. User sends a message/task → `TextMessage` with `source='user'`
2. Assistant processes and responds → `TextMessage` with `source='assistant'`

## Examples

### Example 1: Simple Question-Answer

This example demonstrates the most straightforward way to interact with an AssistantAgent using the high-level `run()` method.

In [ ]:
! pip install --quiet -U "autogen-agentchat>=0.7" "autogen-ext[openai]>=0.7" rich

In [ ]:
from rich import print as rprint
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def main() -> None:
    # Initialize the model client
    model_client = OpenAIChatCompletionClient(model="gpt-4o")

    # Create an AssistantAgent
    agent = AssistantAgent("assistant", model_client=model_client)
    
    # Execute a task using the high-level run() method
    task_result = await agent.run(task="Say 'Hello World!' in five languages.")
    
    # Display the results
    rprint(task_result)
    
    # Clean up resources
    await model_client.close()

await main()

#### Understanding the Output

The `run()` method returns a `TaskResult` object containing:

- **messages**: A list of `TextMessage` objects representing the conversation
  - First message (`source='user'`): The initial task/prompt
  - Second message (`source='assistant'`): The agent's response
- **stop_reason**: Information about why the conversation ended (if applicable)

#### Key Components Explained

- **TaskResult** [[API docs](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.base.html#autogen_agentchat.base.TaskResult)]: Container for the complete interaction results
- **TextMessage** [[API docs](https://microsoft.github.io/autogen/stable/reference/python/autogen_agentchat.messages.html#autogen_agentchat.messages.TextMessage)]: Individual message in the conversation

#### Message Attributes

Each `TextMessage` contains:
- `id`: Unique identifier for the message
- `source`: Origin of the message (`'user'` or `'assistant'`)
- `content`: The actual message text
- `models_usage`: Token usage information (for assistant messages)
- `created_at`: Timestamp of message creation
- `metadata`: Additional message metadata

### Message Source Pattern

Observe the `source` attribute for the messages:

- **First TextMessage (source='user')**: This represents the initial task/prompt that was given to the agent. When you call `await agent.run(task="Say 'Hello World!' in five languages.")`, AutoGen creates a TextMessage with the task content and marks it as coming from the 'user' source.
- **Second TextMessage (source='assistant')**: This represents the response generated by the AssistantAgent. The agent processes the user's task and generates a response, which becomes a TextMessage with source='assistant'.

This pattern follows the typical conversational flow:
- User sends a message/task → TextMessage with source='user'
- Assistant responds → TextMessage with source='assistant'

### Example 2: Lower-Level API Usage

This example shows how to use the lower-level `on_messages()` method for more granular control over message handling.

In [ ]:
from rich import print as rprint
from autogen_core import CancellationToken
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def main() -> None:
    # Initialize the model client
    model_client = OpenAIChatCompletionClient(model="gpt-4o")

    # Create an AssistantAgent
    agent = AssistantAgent("assistant", model_client=model_client)
    
    # Create a message manually
    user_message = TextMessage(
        content="Say 'Hello World!' in five languages.", 
        source="user"
    )
    
    # Process the message using the lower-level API
    response = await agent.on_messages([user_message], CancellationToken())
    
    # Display the response
    rprint(response)
    
    # Clean up resources
    await model_client.close()

await main()

#### Key Differences from High-Level API

1. **Manual Message Creation**: You must create `TextMessage` objects explicitly
2. **Response vs TaskResult**: Returns a `Response` object instead of `TaskResult`
3. **Single Response**: Only returns the agent's response, not the full conversation history
4. **Cancellation Support**: Accepts a `CancellationToken` for operation cancellation

#### Understanding the Response Object

The `Response` object contains:
- **chat_message**: The single `TextMessage` response from the assistant
- **inner_messages**: List of internal messages (empty for simple interactions)

#### Frequently Asked Questions

**Q: Why is there only one chat_message?**
A: Unlike `run()` which returns the full conversation, `on_messages()` returns only the agent's response to your input messages.

**Q: What are inner_messages? Why is it empty?**
A: Inner messages contain internal processing steps or tool calls that occurred during response generation. For simple text responses, this list is typically empty.

## Method Comparison

| Feature | `run()` | `on_messages()` | `on_messages_stream()` |
|---------|---------|-----------------|------------------------|
| **Ease of Use** | High-level, simple | Lower-level, more control | Lower-level, streaming |
| **Input** | String task | List of TextMessage objects | List of TextMessage objects |
| **Output** | TaskResult with full conversation | Response with single message | AsyncIterator of Response objects |
| **Use Case** | Quick tasks, simple interactions | Complex flows, message manipulation | Real-time streaming, long responses |
| **Cancellation** | Not directly supported | Supports CancellationToken | Supports CancellationToken |
| **Response Type** | Complete response at once | Complete response at once | Incremental streaming chunks |

### Note on `on_messages_stream()`

**`on_messages_stream()`** is particularly useful when:
- You need to display responses as they're being generated (real-time UI updates)
- Working with long responses that benefit from progressive display
- Building interactive chat interfaces where users see typing indicators
- Processing very long outputs where you want to start handling partial responses immediately
- Implementing token-by-token or chunk-by-chunk processing workflows

The streaming method returns an `AsyncIterator` that yields `Response` objects incrementally, allowing you to process and display content as it becomes available rather than waiting for the complete response.

This would fit well in your best practices section as well, helping users understand when streaming is beneficial versus the standard synchronous approaches.

## Best Practices

### When to Use Each Method

- **Use `run()`** for:
  - Simple, one-off tasks
  - Quick prototyping
  - When you need the full conversation history
  - Batch processing where you can wait for complete responses

- **Use `on_messages()`** for:
  - Building complex conversational flows
  - When you need fine-grained control over messages
  - Implementing custom message handling logic
  - When cancellation support is required
  - Processing responses that don't require real-time feedback

- **Use `on_messages_stream()`** for:
  - Interactive chat interfaces where users expect real-time responses
  - Long-form content generation (stories, articles, code) where progressive display improves UX
  - Applications requiring immediate feedback or "typing" indicators
  - Processing very large responses where you want to start handling partial content
  - Token-by-token analysis or real-time content filtering
  - Building responsive UI experiences that feel more natural and engaging
  - Scenarios where network latency benefits from chunked delivery
  - Applications that need to provide user interruption capabilities during generation

### Always remember to close the model client to free up resources:

In [ ]:
# Example of proper resource management
async def example_with_cleanup():
    model_client = OpenAIChatCompletionClient(model="gpt-4o")
    try:
        agent = AssistantAgent("assistant", model_client=model_client)
        task_result = await agent.run(task="Your task here")
        return task_result
    except Exception as e:
        print(f"Error occurred: {e}")
        # Handle error appropriately
        raise
    finally:
        await model_client.close()

## Next Steps

This tutorial covered the basics of AutoGen AutoChat agents. To continue learning:

1. **Explore multi-agent conversations**: Learn how agents can communicate with each other
2. **Learn about tool integration**: Discover how to give agents access to external tools and APIs
3. **Implement custom agent behaviors**: Create specialized agents for specific use cases
4. **Study advanced message types**: Explore beyond TextMessage for richer interactions

For more advanced topics, refer to the [AutoGen documentation](https://microsoft.github.io/autogen/) and explore the various agent types and capabilities available in the framework.

### Additional Resources

- [AutoGen GitHub Repository](https://github.com/microsoft/autogen)
- [API Reference Documentation](https://microsoft.github.io/autogen/stable/reference/python/)
- [Community Examples and Tutorials](https://microsoft.github.io/autogen/stable/user-guide/)